# 05 — Modelos supervisados para detección y predicción de aperturas/cierres bruscos (1-min)

## Propósito
Construir y evaluar modelos supervisados, con datos muestreados a $\Delta t \approx 1$ minuto, para:

**Detección (nowcasting)**  
Identificar el minuto de inicio de:
- apertura,
- cierre,
- apertura brusca,
- cierre brusco.

**Predicción (forecast probabilístico)**  
Estimar la probabilidad de que ocurra un inicio brusco dentro de un horizonte $K$:
$$
\mathbb{P}\big(\text{inicio brusco en } (t, t+K]\big),\qquad K\in\{10,30,60\}\ \text{min}
$$

---

## Restricción metodológica: evitar *leakage*
Los *features* en el instante $t$ usan solo historia $\le t$ (ventanas hacia atrás).  
Los *labels* pueden mirar futuro (verdad terreno), pero no deben filtrarse en los *features*.

---

## Métricas (dos escalas)

### A) Por minuto
- **PR-AUC** (Average Precision): adecuada para clases desbalanceadas.
- **Prevalencia** en test y **lift** $=\text{AP}/\text{prevalencia}$ como referencia.

### B) Por evento (operacional)
- **Detección de start**: un evento con inicio $t_0$ está “detectado” si existe una alarma en $[t_0-1,\,t_0+1]$.
- **Forecast**: un evento con inicio $t_0$ está “anticipado” si existe una alarma en $[t_0-K,\,t_0-1]$.
- Se reporta además el **lead time** (minutos entre la primera alarma y $t_0$).

---

## Notas importantes
1) **Consistencia label ↔ evaluación por evento**:  
   - `y_open_start_win` y `y_close_start` representan **todos** los starts (no solo una subfamilia).  
   - La evaluación por evento usa la misma definición: **lista completa** de starts del tipo correspondiente.  
   - Para targets “bruscos”, tanto label como evaluación usan solo los starts bruscos.
2) **Muestreo 1-min**: para aperturas se emplea una ventana de label en $[t_0-1, t_0+1]$ para compensar desfases posibles.


In [73]:
from pathlib import Path
import numpy as np
import pandas as pd

from IPython.display import display

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
import joblib

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

PROJECT_ROOT = Path('/home/maxi/datascience_esval')

SENS_FILE   = PROJECT_ROOT / 'data' / 'processed' / 'sensores_1min_clean.parquet'
EVENTS_FILE = PROJECT_ROOT / 'data' / 'processed' / 'eventos_apertura_cierre.parquet'

OUT_FEATS   = PROJECT_ROOT / 'data' / 'processed' / 'ml_features_1min.parquet'
OUT_LABELS  = PROJECT_ROOT / 'data' / 'processed' / 'ml_labels_1min.parquet'
OUT_REPORT  = PROJECT_ROOT / 'data' / 'processed' / 'ml_report_metrics.parquet'
OUT_SENSIT  = PROJECT_ROOT / 'data' / 'processed' / 'ml_sensitivity_budgets.parquet'

MODELS_DIR  = PROJECT_ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

SENS_FILE.exists(), EVENTS_FILE.exists()


(True, True)

## 1) Carga de serie 1-min y eventos

In [74]:
sens = pd.read_parquet(SENS_FILE)
events = pd.read_parquet(EVENTS_FILE)

# Índice temporal
if not isinstance(sens.index, pd.DatetimeIndex):
    time_candidates = [c for c in sens.columns if ('fecha' in c.lower()) or ('time' in c.lower()) or ('hora' in c.lower())]
    if not time_candidates:
        raise ValueError("No se encontró índice temporal en sensores.")
    tcol = time_candidates[0]
    sens[tcol] = pd.to_datetime(sens[tcol])
    sens = sens.set_index(tcol)

sens = sens.sort_index()
events['start'] = pd.to_datetime(events['start'])
events['end']   = pd.to_datetime(events['end'])

def pick_col(cols, keys):
    for k in keys:
        for c in cols:
            if k in c.lower():
                return c
    return None

Q_COL = next((c for c in sens.columns if c.lower() == 'q_lps'), None) or pick_col(sens.columns, ['q_lps','caudal','flow'])
P_COL = next((c for c in sens.columns if c.lower() == 'p_bar'), None) or pick_col(sens.columns, ['p_bar','pres','pe_','booster'])

Q_COL, P_COL, sens.shape, sens.index.min(), sens.index.max()


('q_lps',
 'p_bar',
 (475201, 9),
 Timestamp('2025-01-01 00:00:00'),
 Timestamp('2025-11-27 00:00:00'))

## 2) Definición de *starts* por evento (verdad terreno)

Se define “brusco” usando, en orden preferente:
- `rise_bin_4min == '0-4'`, o
- `rise_time_min <= 1`, o
- `max_abs_dQ_1min_lps >= 16`.


In [75]:
SPLIT_DATE = pd.Timestamp('2025-10-01 00:00:00')
ev = events.copy()

# Heurística de brusquedad
if 'rise_bin_4min' in ev.columns:
    ev['is_brusco'] = (ev['rise_bin_4min'].astype(str) == '0-4')
elif 'rise_time_min' in ev.columns:
    ev['is_brusco'] = (ev['rise_time_min'] <= 1)
elif 'max_abs_dQ_1min_lps' in ev.columns:
    ev['is_brusco'] = (ev['max_abs_dQ_1min_lps'] >= 16)
else:
    ev['is_brusco'] = False

def event_starts(ev, typ, brusco=None):
    s = ev[ev['type'] == typ].copy()
    if brusco is True:
        s = s[s['is_brusco']]
    if brusco is False:
        s = s[~s['is_brusco']]
    return s['start'].dt.floor('min').sort_values().reset_index(drop=True)

starts_all = {
    'open_all':  event_starts(ev, 'apertura', brusco=None),
    'open_bru':  event_starts(ev, 'apertura', brusco=True),
    'close_all': event_starts(ev, 'cierre',   brusco=None),
    'close_bru': event_starts(ev, 'cierre',   brusco=True),
}

def split_starts(s):
    tr = s[s < SPLIT_DATE].reset_index(drop=True)
    te = s[s >= SPLIT_DATE].reset_index(drop=True)
    return tr, te

starts_split = {k: split_starts(v) for k, v in starts_all.items()}

pd.DataFrame({
    'key': list(starts_all.keys()),
    'n_total': [len(starts_all[k]) for k in starts_all],
    'n_train': [len(starts_split[k][0]) for k in starts_all],
    'n_test':  [len(starts_split[k][1]) for k in starts_all],
}).sort_values('key')


,key,n_total,n_train,n_test
2,close_all,1555,1252,303
3,close_bru,1483,1191,292
0,open_all,1103,887,216
1,open_bru,1004,794,210


## 3) Labels por minuto

- **Aperturas (start)**: ventana de label en $[t_0-1, t_0+1]$.
- **Cierres (start)**: start exacto (un minuto).
- **Forecast** (`nextK`): 1 si existe un inicio brusco en $(t, t+K]$.

La evaluación por evento usa siempre los *starts* reales (listas `starts_*_test`).


In [76]:
idx = sens.index
labels = pd.DataFrame(index=idx)

# Start labels
labels['y_open_start_win'] = 0
labels['y_open_brusco_start_win'] = 0
labels['y_close_start'] = 0
labels['y_close_brusco_start'] = 0

def mark_window(index, times, w_before=1, w_after=1):
    # devuelve máscara booleana con unión de ventanas
    mask = np.zeros(len(index), dtype=bool)
    index = pd.DatetimeIndex(index)
    for t0 in pd.DatetimeIndex(times):
        w0 = t0 - pd.Timedelta(minutes=w_before)
        w1 = t0 + pd.Timedelta(minutes=w_after)
        lo = index.searchsorted(w0, side='left')
        hi = index.searchsorted(w1, side='right')
        if hi > lo:
            mask[lo:hi] = True
    return mask

# Aperturas: ventana
m_open_all = mark_window(labels.index, starts_all['open_all'], w_before=1, w_after=1)
m_open_bru = mark_window(labels.index, starts_all['open_bru'], w_before=1, w_after=1)
labels.loc[m_open_all, 'y_open_start_win'] = 1
labels.loc[m_open_bru, 'y_open_brusco_start_win'] = 1

# Cierres: exacto
close_exact_all = starts_all['close_all'][starts_all['close_all'].isin(labels.index)]
close_exact_bru = starts_all['close_bru'][starts_all['close_bru'].isin(labels.index)]
labels.loc[close_exact_all.values, 'y_close_start'] = 1
labels.loc[close_exact_bru.values, 'y_close_brusco_start'] = 1

# Forecast nextK
Ks = [10, 30, 60]
for K in Ks:
    # open: desde label brusco en ventana
    s = labels['y_open_brusco_start_win'].astype(float)
    rev = s[::-1]
    labels[f'y_open_brusco_next{K}'] = ((rev.rolling(K, min_periods=1).max().shift(1)).fillna(0)[::-1] > 0).astype(int)

    # close: desde start exacto brusco
    s = labels['y_close_brusco_start'].astype(float)
    rev = s[::-1]
    labels[f'y_close_brusco_next{K}'] = ((rev.rolling(K, min_periods=1).max().shift(1)).fillna(0)[::-1] > 0).astype(int)

labels.sum().to_dict()


{'y_open_start_win': 3309,
 'y_open_brusco_start_win': 3012,
 'y_close_start': 1555,
 'y_close_brusco_start': 1483,
 'y_open_brusco_next10': 11918,
 'y_close_brusco_next10': 14803,
 'y_open_brusco_next30': 30085,
 'y_close_brusco_next30': 41950,
 'y_open_brusco_next60': 53593,
 'y_close_brusco_next60': 73759}

## 4) Ingeniería de *features* (solo pasado)

Features en $t$:
- $Q(t)$, $P_g(t)$
- $\Delta Q(t)$, $\Delta P(t)$ (1-min)
- rolling hacia atrás $w\in\{5,15,60\}$: mean/std/min/max
- máximos de $|\Delta Q|$, $|\Delta P|$ en ventana
- calendario: seno/coseno (hora y día de semana)

Los NaN de `diff()`/`rolling()` se imputan con mediana (ajustada solo con train).


In [77]:
df = sens[[Q_COL, P_COL]].copy().rename(columns={Q_COL:'Q_lps', P_COL:'P_bar'})

df['dQ_1'] = df['Q_lps'].diff()
df['dP_1'] = df['P_bar'].diff()

t = df.index
df['hour'] = t.hour
df['dow']  = t.dayofweek
df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
df['dow_sin']  = np.sin(2*np.pi*df['dow']/7)
df['dow_cos']  = np.cos(2*np.pi*df['dow']/7)

windows = [5, 15, 60]
for w in windows:
    rQ  = df['Q_lps'].rolling(w, min_periods=max(2, w//3))
    rP  = df['P_bar'].rolling(w, min_periods=max(2, w//3))
    rdQ = df['dQ_1'].abs().rolling(w, min_periods=max(2, w//3))
    rdP = df['dP_1'].abs().rolling(w, min_periods=max(2, w//3))

    df[f'Q_mean_{w}'] = rQ.mean()
    df[f'Q_std_{w}']  = rQ.std()
    df[f'Q_min_{w}']  = rQ.min()
    df[f'Q_max_{w}']  = rQ.max()

    df[f'P_mean_{w}'] = rP.mean()
    df[f'P_std_{w}']  = rP.std()
    df[f'P_min_{w}']  = rP.min()
    df[f'P_max_{w}']  = rP.max()

    df[f'max_abs_dQ_{w}'] = rdQ.max()
    df[f'max_abs_dP_{w}'] = rdP.max()

base = ['Q_lps','P_bar','dQ_1','dP_1','hour_sin','hour_cos','dow_sin','dow_cos']
auto = [c for c in df.columns
        if (c.startswith('Q_') or c.startswith('P_') or c.startswith('max_abs_'))
        and c not in ['Q_lps','P_bar']]

feat_cols = list(dict.fromkeys(base + auto))
X = df[feat_cols].copy()
X = X.loc[:, ~X.columns.duplicated()].copy()
assert X.columns.is_unique

data = X.join(labels, how='left')
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna(subset=['Q_lps','P_bar'])

train = data[data.index < SPLIT_DATE].copy()
test  = data[data.index >= SPLIT_DATE].copy()

(len(train), len(test))


(384847, 77283)

## 5) Modelos

- Regresión logística (**LogReg**): interpretable.
- HistGradientBoosting (**HGB**): no lineal.

Se limita el tamaño de entrenamiento por target para controlar costo computacional.


In [78]:
MAX_TRAIN_SAMPLES = 200_000

def make_train_xy(df_train, target, neg_ratio=20, random_state=42, max_samples=MAX_TRAIN_SAMPLES):
    y = df_train[target].astype(int)
    X_ = df_train[feat_cols].astype(float)

    pos_idx = y[y==1].index
    neg_idx = y[y==0].index
    if len(pos_idx) == 0:
        raise ValueError(f'No hay positivos para {target} en train.')

    pos_budget = max(1, int(max_samples / (neg_ratio + 1)))
    if len(pos_idx) > pos_budget:
        pos_idx = pd.Index(resample(pos_idx.to_series(), replace=False, n_samples=pos_budget, random_state=random_state).values)

    n_neg = min(len(neg_idx), neg_ratio * len(pos_idx))
    neg_budget = max_samples - len(pos_idx)
    n_neg = min(n_neg, max(0, neg_budget))

    if n_neg > 0:
        neg_idx_s = resample(neg_idx.to_series(), replace=False, n_samples=n_neg, random_state=random_state).values
        idx = pos_idx.union(pd.Index(neg_idx_s))
    else:
        idx = pos_idx

    return X_.loc[idx], y.loc[idx]

def train_models_for_target(target, neg_ratio=20, random_state=42):
    Xtr, ytr = make_train_xy(train, target, neg_ratio=neg_ratio, random_state=random_state)

    logreg = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler(with_centering=True)),
        ('clf', LogisticRegression(max_iter=2000, class_weight='balanced', solver='lbfgs'))
    ])
    logreg.fit(Xtr.values, ytr.values)

    hgb = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('clf', HistGradientBoostingClassifier(max_depth=6, learning_rate=0.08, max_iter=300, random_state=random_state))
    ])
    hgb.fit(Xtr.values, ytr.values)

    return logreg, hgb


## 6) Selección de umbral con barrido en cola alta

Se incluye la cola alta de probabilidades (0.995–0.9999) para controlar presupuestos de alertas.


In [79]:
def sweep_thresholds_tail(proba, n_mid=140, n_tail=120):
    proba = np.asarray(proba)
    proba = proba[np.isfinite(proba)]
    if len(proba) == 0:
        return np.array([0.5])

    qs_mid = np.linspace(0.01, 0.99, n_mid)
    qs_tail = np.array([0.995, 0.997, 0.999, 0.9995, 0.9997, 0.9999])
    th_q = np.unique(np.quantile(proba, np.unique(np.r_[qs_mid, qs_tail])))

    q99 = float(np.quantile(proba, 0.99))
    mx = float(np.max(proba))
    if mx > q99:
        th_tail = np.unique(np.linspace(q99, mx, n_tail))
        ths = np.unique(np.r_[th_q, th_tail])
    else:
        ths = th_q

    ths = ths[(ths >= 0.0) & (ths <= 1.0)]
    if len(ths) == 0:
        ths = np.array([0.5])
    return np.unique(ths)

def pr_auc(y_true, proba):
    return float(average_precision_score(y_true, proba))

def days_span(index):
    return max(int((index.max() - index.min()).days), 1)

def start_alert_accounting(pred_times, starts, tol_min=1):
    pred_times = pd.DatetimeIndex(pred_times).sort_values()
    starts = pd.DatetimeIndex(starts).sort_values()

    if len(pred_times) == 0:
        return dict(n_alerts=0, n_tp_alerts=0, n_fp_alerts=0, precision_alerts=np.nan)

    tp_mask = np.zeros(len(pred_times), dtype=bool)
    for t0 in starts:
        w0 = t0 - pd.Timedelta(minutes=tol_min)
        w1 = t0 + pd.Timedelta(minutes=tol_min)
        lo = pred_times.searchsorted(w0, side='left')
        hi = pred_times.searchsorted(w1, side='right')
        if hi > lo:
            tp_mask[lo:hi] = True

    n_alerts = len(pred_times)
    n_tp = int(tp_mask.sum())
    n_fp = int(n_alerts - n_tp)
    prec = float(n_tp / n_alerts) if n_alerts else np.nan
    return dict(n_alerts=n_alerts, n_tp_alerts=n_tp, n_fp_alerts=n_fp, precision_alerts=prec)

def event_recall_start(starts, pred_times, tol_min=1):
    pred_times = pd.DatetimeIndex(pred_times).sort_values()
    starts = pd.DatetimeIndex(starts).sort_values()
    hit = 0
    for t0 in starts:
        w0 = t0 - pd.Timedelta(minutes=tol_min)
        w1 = t0 + pd.Timedelta(minutes=tol_min)
        lo = pred_times.searchsorted(w0, side='left')
        hi = pred_times.searchsorted(w1, side='right')
        if hi > lo:
            hit += 1
    n = len(starts)
    return int(n), int(hit), float(hit / max(n, 1))

def forecast_anticipation(starts, pred_times, K):
    pred_times = pd.DatetimeIndex(pred_times).sort_values()
    starts = pd.DatetimeIndex(starts).sort_values()
    hits = 0
    leads = []
    for t0 in starts:
        w0 = t0 - pd.Timedelta(minutes=K)
        w1 = t0 - pd.Timedelta(minutes=1)
        lo = pred_times.searchsorted(w0, side='left')
        hi = pred_times.searchsorted(w1, side='right')
        if hi > lo:
            hits += 1
            first_alarm = pred_times[lo]
            leads.append(float((t0 - first_alarm).total_seconds()/60.0))
    n = len(starts)
    rec = hits / max(n, 1)
    lead_med = float(np.nanmedian(leads)) if leads else np.nan
    lead_p10 = float(np.nanpercentile(leads, 10)) if leads else np.nan
    lead_p90 = float(np.nanpercentile(leads, 90)) if leads else np.nan
    return int(n), int(hits), float(rec), lead_med, lead_p10, lead_p90

def best_thr_start(test_index, proba, starts_test, fp_day_max=12.0, tol_min=1):
    ths = sweep_thresholds_tail(proba)
    days = days_span(test_index)
    best = None

    for th in ths:
        pred_times = test_index[proba >= th]
        acc = start_alert_accounting(pred_times, starts_test, tol_min=tol_min)
        fpday = acc['n_fp_alerts'] / days
        if fpday > fp_day_max:
            continue
        n_ev, hit, r_ev = event_recall_start(starts_test, pred_times, tol_min=tol_min)
        cand = (r_ev, -fpday, -acc['n_alerts'], th)
        if (best is None) or (cand > best[0]):
            best = (cand, float(th), float(fpday), acc, n_ev, hit, float(r_ev), True)

    if best is None:
        th = float(np.max(ths))
        pred_times = test_index[proba >= th]
        acc = start_alert_accounting(pred_times, starts_test, tol_min=tol_min)
        fpday = acc['n_fp_alerts'] / days
        n_ev, hit, r_ev = event_recall_start(starts_test, pred_times, tol_min=tol_min)
        return th, fpday, acc, n_ev, hit, r_ev, False

    _, th, fpday, acc, n_ev, hit, r_ev, ok = best
    return th, fpday, acc, n_ev, hit, r_ev, ok

def best_thr_forecast(test_index, proba, starts_test, K, alerts_day_max=50.0):
    ths = sweep_thresholds_tail(proba)
    days = days_span(test_index)
    best = None

    for th in ths:
        pred_times = test_index[proba >= th]
        aday = len(pred_times) / days
        if aday > alerts_day_max:
            continue
        n, hits, rec, lead_med, lead_p10, lead_p90 = forecast_anticipation(starts_test, pred_times, K)
        cand = (rec, -aday, th)
        if (best is None) or (cand > best[0]):
            best = (cand, float(th), float(aday), int(len(pred_times)), n, hits, rec, lead_med, lead_p10, lead_p90, True)

    if best is None:
        th = float(np.max(ths))
        pred_times = test_index[proba >= th]
        aday = len(pred_times) / days
        n, hits, rec, lead_med, lead_p10, lead_p90 = forecast_anticipation(starts_test, pred_times, K)
        return th, float(aday), int(len(pred_times)), n, hits, rec, lead_med, lead_p10, lead_p90, False

    _, th, aday, n_alerts, n, hits, rec, lead_med, lead_p10, lead_p90, ok = best
    return th, aday, n_alerts, n, hits, rec, lead_med, lead_p10, lead_p90, ok


## 7) Entrenamiento, evaluación y reporte

Política base:
- Start: $FP/\text{día}\le FP\_DAY\_MAX$
- Forecast: alertas/día $\le ALERTS\_DAY\_MAX$


In [80]:
FP_DAY_MAX = 12.0
ALERTS_DAY_MAX = 50.0
TOL_MIN = 1

targets_start = [
    ('y_open_start_win',         'open_all'),
    ('y_open_brusco_start_win',  'open_bru'),
    ('y_close_start',            'close_all'),
    ('y_close_brusco_start',     'close_bru'),
]

targets_forecast = []
for K in [10,30,60]:
    targets_forecast.append((f'y_open_brusco_next{K}',  'open_bru',  K))
    targets_forecast.append((f'y_close_brusco_next{K}', 'close_bru', K))

Xte = test[feat_cols].astype(float).values
test_index = test.index

trained = {}
rows = []

def fit_eval_start(target_col, starts_key, neg_ratio=30, random_state=42):
    logreg, hgb = train_models_for_target(target_col, neg_ratio=neg_ratio, random_state=random_state)
    trained[(target_col,'logreg')] = logreg
    trained[(target_col,'hgb')] = hgb

    y_true = test[target_col].astype(int).values
    prev = float(np.mean(y_true))

    starts_te = pd.DatetimeIndex(starts_split[starts_key][1])

    for name, model in [('logreg', logreg), ('hgb', hgb)]:
        proba = model.predict_proba(Xte)[:,1]
        ap = pr_auc(y_true, proba)
        lift = (ap / prev) if prev > 0 else np.nan

        th, fpday, acc, n_ev, hit, r_ev, ok = best_thr_start(
            test_index, proba, starts_te, fp_day_max=FP_DAY_MAX, tol_min=TOL_MIN
        )
        pred_times = test_index[proba >= th]
        aday = len(pred_times) / days_span(test_index)

        rows.append({
            'task': 'start',
            'target': target_col,
            'starts_key': starts_key,
            'model': name,
            'AP(PR-AUC)': float(ap),
            'prevalence_test': float(prev),
            'lift_AP_vs_prev': float(lift) if np.isfinite(lift) else np.nan,
            'thr': float(th),
            'policy_met': bool(ok),
            'FP_per_day': float(fpday),
            'alerts_per_day': float(aday),
            'n_alerts': int(len(pred_times)),
            'precision_alerts': float(acc['precision_alerts']) if acc['n_alerts'] > 0 else np.nan,
            'n_events_test': int(n_ev),
            'event_hits': int(hit),
            'event_recall': float(r_ev),
            'K_min': np.nan,
            'lead_med_min': np.nan,
            'lead_p10_min': np.nan,
            'lead_p90_min': np.nan,
        })

def fit_eval_forecast(target_col, starts_key, K, neg_ratio=20, random_state=42):
    logreg, hgb = train_models_for_target(target_col, neg_ratio=neg_ratio, random_state=random_state)
    trained[(target_col,'logreg')] = logreg
    trained[(target_col,'hgb')] = hgb

    y_true = test[target_col].astype(int).values
    prev = float(np.mean(y_true))

    starts_te = pd.DatetimeIndex(starts_split[starts_key][1])

    for name, model in [('logreg', logreg), ('hgb', hgb)]:
        proba = model.predict_proba(Xte)[:,1]
        ap = pr_auc(y_true, proba)
        lift = (ap / prev) if prev > 0 else np.nan

        th, aday, n_alerts, n_ev, hit, rec, lead_med, lead_p10, lead_p90, ok = best_thr_forecast(
            test_index, proba, starts_te, K=int(K), alerts_day_max=ALERTS_DAY_MAX
        )

        rows.append({
            'task': 'forecast',
            'target': target_col,
            'starts_key': starts_key,
            'model': name,
            'AP(PR-AUC)': float(ap),
            'prevalence_test': float(prev),
            'lift_AP_vs_prev': float(lift) if np.isfinite(lift) else np.nan,
            'thr': float(th),
            'policy_met': bool(ok),
            'FP_per_day': np.nan,
            'alerts_per_day': float(aday),
            'n_alerts': int(n_alerts),
            'precision_alerts': np.nan,
            'n_events_test': int(n_ev),
            'event_hits': int(hit),
            'event_recall': float(rec),
            'K_min': float(K),
            'lead_med_min': float(lead_med),
            'lead_p10_min': float(lead_p10),
            'lead_p90_min': float(lead_p90),
        })

for target_col, starts_key in targets_start:
    fit_eval_start(target_col, starts_key, neg_ratio=30)

for target_col, starts_key, K in targets_forecast:
    fit_eval_forecast(target_col, starts_key, K, neg_ratio=20)

report = pd.DataFrame(rows).sort_values(['task','target','model']).reset_index(drop=True)
report


,task,target,starts_key,model,AP(PR-AUC),prevalence_test,lift_AP_vs_prev,thr,policy_met,FP_per_day,alerts_per_day,n_alerts,precision_alerts,n_events_test,event_hits,event_recall,K_min,lead_med_min,lead_p10_min,lead_p90_min
0,forecast,y_close_brusco_next10,close_bru,hgb,0.412412,0.036994,11.148103,0.303729,True,NaN,42.245614,2408,NaN,292,265,0.907534,10.0,5.0,2.0,10.0
1,forecast,y_close_brusco_next10,close_bru,logreg,0.365955,0.036994,9.892310,0.874504,True,NaN,42.245614,2408,NaN,292,276,0.945205,10.0,5.0,2.0,9.0
2,forecast,y_close_brusco_next30,close_bru,hgb,0.553921,0.102532,5.402404,0.480635,True,NaN,42.245614,2408,NaN,292,236,0.808219,30.0,14.5,3.0,30.0
3,forecast,y_close_brusco_next30,close_bru,logreg,0.443402,0.102532,4.324510,0.913835,True,NaN,42.245614,2408,NaN,292,265,0.907534,30.0,6.0,3.0,28.0
4,forecast,y_close_brusco_next60,close_bru,hgb,0.694515,0.173518,4.002549,0.704210,True,NaN,42.245614,2408,NaN,292,204,0.698630,60.0,41.0,4.0,60.0
5,forecast,y_close_brusco_next60,close_bru,logreg,0.586458,0.173518,3.379810,0.938372,True,NaN,42.245614,2408,NaN,292,220,0.753425,60.0,30.0,3.0,60.0
6,forecast,y_open_brusco_next10,open_bru,hgb,0.169779,0.031171,5.446674,0.260104,True,NaN,42.245614,2408,NaN,210,74,0.352381,10.0,10.0,2.3,10.0
7,forecast,y_open_brusco_next10,open_bru,logreg,0.103058,0.031171,3.306210,0.864027,True,NaN,42.245614,2408,NaN,210,33,0.157143,10.0,10.0,2.0,10.0
8,forecast,y_open_brusco_next30,open_bru,hgb,0.364528,0.077042,4.731580,0.342126,True,NaN,42.245614,2408,NaN,210,106,0.504762,30.0,23.0,4.0,30.0
9,forecast,y_open_brusco_next30,open_bru,logreg,0.242123,0.077042,3.142763,0.871055,True,NaN,42.245614,2408,NaN,210,43,0.204762,30.0,28.0,9.2,30.0


## 8) Sensibilidad a presupuestos

- Start: $FP/\text{día}\in\{1,3,5,10,12,20\}$
- Forecast: alertas/día $\in\{5,10,20,50\}$

Se reporta para modelos HGB como referencia.


In [81]:
FP_BUDGETS = [1, 3, 5, 10, 12, 20]
ALERT_BUDGETS = [5, 10, 20, 50]

def sensitivity_start(target_col, starts_key, model_name='hgb'):
    logreg, hgb = train_models_for_target(target_col, neg_ratio=30, random_state=42)
    model = hgb if model_name == 'hgb' else logreg

    y_true = test[target_col].astype(int).values
    ap = pr_auc(y_true, model.predict_proba(Xte)[:,1])
    prev = float(np.mean(y_true))

    starts_te = pd.DatetimeIndex(starts_split[starts_key][1])
    proba = model.predict_proba(Xte)[:,1]
    days = days_span(test_index)

    out = []
    for fpmax in FP_BUDGETS:
        th, fpday, acc, n_ev, hit, r_ev, ok = best_thr_start(
            test_index, proba, starts_te, fp_day_max=float(fpmax), tol_min=TOL_MIN
        )
        pred_times = test_index[proba >= th]
        out.append({
            'task': 'start',
            'target': target_col,
            'starts_key': starts_key,
            'model': model_name,
            'AP(PR-AUC)': float(ap),
            'prevalence_test': float(prev),
            'FP_DAY_MAX': float(fpmax),
            'thr': float(th),
            'policy_met': bool(ok),
            'FP_per_day': float(fpday),
            'alerts_per_day': float(len(pred_times)/days),
            'n_alerts': int(len(pred_times)),
            'event_recall': float(r_ev),
            'precision_alerts': float(acc['precision_alerts']) if acc['n_alerts'] > 0 else np.nan,
        })
    return pd.DataFrame(out)

def sensitivity_forecast(target_col, starts_key, K, model_name='hgb'):
    logreg, hgb = train_models_for_target(target_col, neg_ratio=20, random_state=42)
    model = hgb if model_name == 'hgb' else logreg

    y_true = test[target_col].astype(int).values
    ap = pr_auc(y_true, model.predict_proba(Xte)[:,1])
    prev = float(np.mean(y_true))

    starts_te = pd.DatetimeIndex(starts_split[starts_key][1])
    proba = model.predict_proba(Xte)[:,1]

    out = []
    for amax in ALERT_BUDGETS:
        th, aday, n_alerts, n_ev, hit, rec, lead_med, lead_p10, lead_p90, ok = best_thr_forecast(
            test_index, proba, starts_te, K=int(K), alerts_day_max=float(amax)
        )
        out.append({
            'task': 'forecast',
            'target': target_col,
            'starts_key': starts_key,
            'model': model_name,
            'K_min': float(K),
            'AP(PR-AUC)': float(ap),
            'prevalence_test': float(prev),
            'ALERTS_DAY_MAX': float(amax),
            'thr': float(th),
            'policy_met': bool(ok),
            'alerts_per_day': float(aday),
            'n_alerts': int(n_alerts),
            'event_recall': float(rec),
            'lead_med_min': float(lead_med),
            'lead_p10_min': float(lead_p10),
            'lead_p90_min': float(lead_p90),
        })
    return pd.DataFrame(out)

sens_tables = []
sens_tables.append(sensitivity_start('y_close_brusco_start', 'close_bru', model_name='hgb'))
sens_tables.append(sensitivity_start('y_open_brusco_start_win', 'open_bru', model_name='hgb'))
sens_tables.append(sensitivity_forecast('y_close_brusco_next30', 'close_bru', 30, model_name='hgb'))
sens_tables.append(sensitivity_forecast('y_open_brusco_next30',  'open_bru',  30, model_name='hgb'))

sensitivity = pd.concat(sens_tables, ignore_index=True)
sensitivity


,task,target,starts_key,model,AP(PR-AUC),prevalence_test,FP_DAY_MAX,thr,policy_met,FP_per_day,alerts_per_day,n_alerts,event_recall,precision_alerts,K_min,ALERTS_DAY_MAX,lead_med_min,lead_p10_min,lead_p90_min
0,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,1.0,0.909814,True,0.964912,2.666667,152,0.253425,0.638158,NaN,NaN,NaN,NaN,NaN
1,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,3.0,0.829312,True,2.982456,6.789474,387,0.482877,0.560724,NaN,NaN,NaN,NaN,NaN
2,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,5.0,0.747233,True,5.000000,10.596491,604,0.664384,0.528146,NaN,NaN,NaN,NaN,NaN
3,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,10.0,0.671187,True,6.947368,13.561404,773,0.753425,0.487710,NaN,NaN,NaN,NaN,NaN
4,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,12.0,0.671187,True,6.947368,13.561404,773,0.753425,0.487710,NaN,NaN,NaN,NaN,NaN
5,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,20.0,0.372680,True,14.438596,23.122807,1318,0.904110,0.375569,NaN,NaN,NaN,NaN,NaN
6,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,1.0,0.785673,True,0.982456,1.807018,103,0.223810,0.456311,NaN,NaN,NaN,NaN,NaN
7,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,3.0,0.501871,True,3.000000,5.210526,297,0.585714,0.424242,NaN,NaN,NaN,NaN,NaN
8,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,5.0,0.422406,True,4.947368,7.491228,427,0.657143,0.339578,NaN,NaN,NaN,NaN,NaN
9,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,10.0,0.325914,True,9.596491,12.631579,720,0.738095,0.240278,NaN,NaN,NaN,NaN,NaN


## 9) Interpretabilidad (LogReg): coeficientes principales

Se listan coeficientes de mayor magnitud (interpretación descriptiva, no causal) para targets de start.


In [82]:
def top_logreg_features(target, k=20):
    key = (target, 'logreg')
    if key not in trained:
        return None
    model = trained[key]
    clf = model.named_steps['clf']
    coefs = clf.coef_.ravel()
    out = pd.DataFrame({'feature': feat_cols, 'coef': coefs})
    out['abs'] = out['coef'].abs()
    return out.sort_values('abs', ascending=False).head(k)[['feature','coef']]

coef_tables = {
    'y_open_brusco_start_win': top_logreg_features('y_open_brusco_start_win', 20),
    'y_close_brusco_start':    top_logreg_features('y_close_brusco_start', 20),
}
coef_tables


{'y_open_brusco_start_win':       feature      coef
 14    P_min_5 -3.752423
 0       Q_lps -3.220274
 5    hour_cos -2.714476
 15    P_max_5  2.531227
 11    Q_max_5  2.182613
 20   Q_min_15  1.413483
 24   P_min_15 -1.174926
 6     dow_sin  1.173009
 22  P_mean_15  1.052791
 32  P_mean_60  0.804531
 10    Q_min_5 -0.792826
 21   Q_max_15 -0.735352
 8    Q_mean_5  0.686572
 1       P_bar  0.684323
 12   P_mean_5 -0.379311
 18  Q_mean_15  0.374196
 31   Q_max_60  0.311754
 34   P_min_60  0.299278
 25   P_max_15 -0.265155
 13    P_std_5 -0.254603,
 'y_close_brusco_start':           feature       coef
 0           Q_lps  10.516785
 20       Q_min_15  -3.055920
 18      Q_mean_15  -3.017735
 31       Q_max_60  -3.011340
 22      P_mean_15  -3.001827
 25       P_max_15   2.787283
 30       Q_min_60   2.640507
 21       Q_max_15  -2.135282
 8        Q_mean_5   2.013857
 1           P_bar  -1.820952
 28      Q_mean_60  -1.584909
 35       P_max_60   1.551891
 32      P_mean_60  -1.283022
 11

## 10) Exportación de artefactos

In [83]:
OUT_FEATS.parent.mkdir(parents=True, exist_ok=True)

X.to_parquet(OUT_FEATS)
labels.to_parquet(OUT_LABELS)
report.to_parquet(OUT_REPORT)
sensitivity.to_parquet(OUT_SENSIT)

for (tgt, name), model in trained.items():
    path = MODELS_DIR / f'{tgt}__{name}.joblib'
    joblib.dump(model, path)

(OUT_FEATS, OUT_LABELS, OUT_REPORT, OUT_SENSIT, MODELS_DIR)


(PosixPath('/home/maxi/datascience_esval/data/processed/ml_features_1min.parquet'),
 PosixPath('/home/maxi/datascience_esval/data/processed/ml_labels_1min.parquet'),
 PosixPath('/home/maxi/datascience_esval/data/processed/ml_report_metrics.parquet'),
 PosixPath('/home/maxi/datascience_esval/data/processed/ml_sensitivity_budgets.parquet'),
 PosixPath('/home/maxi/datascience_esval/models'))

## 11) Resumen automático

In [84]:
print('=== RESUMEN DATASET (05) ===')
print('Inputs:')
print('SENS_FILE:', SENS_FILE)
print('EVENTS_FILE:', EVENTS_FILE)

print('\nSplit:')
print('SPLIT_DATE:', SPLIT_DATE)
print('train n:', len(train), '| test n:', len(test))

print('\nStarts (n_test):')
for k in ['open_all','open_bru','close_all','close_bru']:
    print(f'{k}:', len(starts_split[k][1]))

print('\nTargets (start):', [t for t,_ in targets_start])
print('Targets (forecast):', [t for t,_,_ in targets_forecast])

print('\nPolítica base:')
print('FP_DAY_MAX (start):', FP_DAY_MAX)
print('ALERTS_DAY_MAX (forecast):', ALERTS_DAY_MAX)
print('TOL_MIN:', TOL_MIN)

print('\nReporte (head):')
display(report.head(25))

print('\nSensibilidad (head):')
display(sensitivity.head(25))

print('\nOutputs:')
print('OUT_FEATS:', OUT_FEATS)
print('OUT_LABELS:', OUT_LABELS)
print('OUT_REPORT:', OUT_REPORT)
print('OUT_SENSIT:', OUT_SENSIT)
print('MODELS_DIR:', MODELS_DIR)
print('=== FIN ===')


=== RESUMEN DATASET (05) ===
Inputs:
SENS_FILE: /home/maxi/datascience_esval/data/processed/sensores_1min_clean.parquet
EVENTS_FILE: /home/maxi/datascience_esval/data/processed/eventos_apertura_cierre.parquet

Split:
SPLIT_DATE: 2025-10-01 00:00:00
train n: 384847 | test n: 77283

Starts (n_test):
open_all: 216
open_bru: 210
close_all: 303
close_bru: 292

Targets (start): ['y_open_start_win', 'y_open_brusco_start_win', 'y_close_start', 'y_close_brusco_start']
Targets (forecast): ['y_open_brusco_next10', 'y_close_brusco_next10', 'y_open_brusco_next30', 'y_close_brusco_next30', 'y_open_brusco_next60', 'y_close_brusco_next60']

Política base:
FP_DAY_MAX (start): 12.0
ALERTS_DAY_MAX (forecast): 50.0
TOL_MIN: 1

Reporte (head):


,task,target,starts_key,model,AP(PR-AUC),prevalence_test,lift_AP_vs_prev,thr,policy_met,FP_per_day,alerts_per_day,n_alerts,precision_alerts,n_events_test,event_hits,event_recall,K_min,lead_med_min,lead_p10_min,lead_p90_min
0,forecast,y_close_brusco_next10,close_bru,hgb,0.412412,0.036994,11.148103,0.303729,True,NaN,42.245614,2408,NaN,292,265,0.907534,10.0,5.0,2.0,10.0
1,forecast,y_close_brusco_next10,close_bru,logreg,0.365955,0.036994,9.892310,0.874504,True,NaN,42.245614,2408,NaN,292,276,0.945205,10.0,5.0,2.0,9.0
2,forecast,y_close_brusco_next30,close_bru,hgb,0.553921,0.102532,5.402404,0.480635,True,NaN,42.245614,2408,NaN,292,236,0.808219,30.0,14.5,3.0,30.0
3,forecast,y_close_brusco_next30,close_bru,logreg,0.443402,0.102532,4.324510,0.913835,True,NaN,42.245614,2408,NaN,292,265,0.907534,30.0,6.0,3.0,28.0
4,forecast,y_close_brusco_next60,close_bru,hgb,0.694515,0.173518,4.002549,0.704210,True,NaN,42.245614,2408,NaN,292,204,0.698630,60.0,41.0,4.0,60.0
5,forecast,y_close_brusco_next60,close_bru,logreg,0.586458,0.173518,3.379810,0.938372,True,NaN,42.245614,2408,NaN,292,220,0.753425,60.0,30.0,3.0,60.0
6,forecast,y_open_brusco_next10,open_bru,hgb,0.169779,0.031171,5.446674,0.260104,True,NaN,42.245614,2408,NaN,210,74,0.352381,10.0,10.0,2.3,10.0
7,forecast,y_open_brusco_next10,open_bru,logreg,0.103058,0.031171,3.306210,0.864027,True,NaN,42.245614,2408,NaN,210,33,0.157143,10.0,10.0,2.0,10.0
8,forecast,y_open_brusco_next30,open_bru,hgb,0.364528,0.077042,4.731580,0.342126,True,NaN,42.245614,2408,NaN,210,106,0.504762,30.0,23.0,4.0,30.0
9,forecast,y_open_brusco_next30,open_bru,logreg,0.242123,0.077042,3.142763,0.871055,True,NaN,42.245614,2408,NaN,210,43,0.204762,30.0,28.0,9.2,30.0



Sensibilidad (head):


,task,target,starts_key,model,AP(PR-AUC),prevalence_test,FP_DAY_MAX,thr,policy_met,FP_per_day,alerts_per_day,n_alerts,event_recall,precision_alerts,K_min,ALERTS_DAY_MAX,lead_med_min,lead_p10_min,lead_p90_min
0,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,1.0,0.909814,True,0.964912,2.666667,152,0.253425,0.638158,NaN,NaN,NaN,NaN,NaN
1,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,3.0,0.829312,True,2.982456,6.789474,387,0.482877,0.560724,NaN,NaN,NaN,NaN,NaN
2,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,5.0,0.747233,True,5.000000,10.596491,604,0.664384,0.528146,NaN,NaN,NaN,NaN,NaN
3,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,10.0,0.671187,True,6.947368,13.561404,773,0.753425,0.487710,NaN,NaN,NaN,NaN,NaN
4,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,12.0,0.671187,True,6.947368,13.561404,773,0.753425,0.487710,NaN,NaN,NaN,NaN,NaN
5,start,y_close_brusco_start,close_bru,hgb,0.345229,0.003778,20.0,0.372680,True,14.438596,23.122807,1318,0.904110,0.375569,NaN,NaN,NaN,NaN,NaN
6,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,1.0,0.785673,True,0.982456,1.807018,103,0.223810,0.456311,NaN,NaN,NaN,NaN,NaN
7,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,3.0,0.501871,True,3.000000,5.210526,297,0.585714,0.424242,NaN,NaN,NaN,NaN,NaN
8,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,5.0,0.422406,True,4.947368,7.491228,427,0.657143,0.339578,NaN,NaN,NaN,NaN,NaN
9,start,y_open_brusco_start_win,open_bru,hgb,0.176434,0.008113,10.0,0.325914,True,9.596491,12.631579,720,0.738095,0.240278,NaN,NaN,NaN,NaN,NaN



Outputs:
OUT_FEATS: /home/maxi/datascience_esval/data/processed/ml_features_1min.parquet
OUT_LABELS: /home/maxi/datascience_esval/data/processed/ml_labels_1min.parquet
OUT_REPORT: /home/maxi/datascience_esval/data/processed/ml_report_metrics.parquet
OUT_SENSIT: /home/maxi/datascience_esval/data/processed/ml_sensitivity_budgets.parquet
MODELS_DIR: /home/maxi/datascience_esval/models
=== FIN ===
